In [2]:
from datetime import date
import json

import openfisca_france
from openfisca_france.model.base import *

#importer numpy permet de coler deux sections de courbes (nécessaire quand on fait varier le salaire de base en-dessous du SMIC mensuel)
import numpy as np

# to debug / trace
from openfisca_core import tools
from openfisca_core.reforms import Reform

# Système socio-fiscal

In [3]:
tax_benefit_system = openfisca_france.FranceTaxBenefitSystem()

# Réforme : 1. Revenu de base par CSG

In [4]:
from numpy import logical_not as not_, minimum as min_, maximum as max_, logical_and as and_, logical_or as or_

In [5]:
#- Hausse de la CSG déductible au taux de 23%
#montant_csg_crds calcul à partir csg.activite.deductible.taux

def modify_legislation_json(reference_legislation_json_copy):
    for value_json in reference_legislation_json_copy['children']['csg']['children']['activite']['children']['deductible']['children']['taux']['values']:
        value_json['value'] = 0.23
    return reference_legislation_json_copy

In [6]:
# Supprimer l'ASPA

class aspa(Variable):
    def function(self, simulation, period):
        period = period.this_month
        return period, self.zeros()

In [7]:
#- Supprimer le RSA
class rsa_socle(Variable):
    def function(self, simulation, period):
        period = period.this_month
        nb_parents = simulation.calculate('nb_parents', period)
        eligib = simulation.calculate('rsa_eligibilite', period)
        rsa_nb_enfants = simulation.calculate('rsa_nb_enfants', period)
        rmi = simulation.legislation_at(period.start).minim.rmi

        nbp = nb_parents + rsa_nb_enfants

        taux = (
            1 +
            (nbp >= 2) * rmi.txp2 +
            (nbp >= 3) * rmi.txp3 +
            (nbp >= 4) * ((nb_parents == 1) * rmi.txps + (nb_parents != 1) * rmi.txp3) +
            max_(nbp - 4, 0) * rmi.txps
        )
        #on met à zéro
        return period, eligib * rmi.rmi * taux * 0

In [8]:
# Suppression des allocations familiales

class af(Variable):
    def function(self, simulation, period):
        period = period.this_month
        af_base = simulation.calculate('af_base', period)
        af_majoration = simulation.calculate('af_majoration', period)
        af_allocation_forfaitaire = simulation.calculate('af_allocation_forfaitaire', period)

        #return period, af_base + af_majoration + af_allocation_forfaitaire
        return period, af_base * 0

In [9]:
# Suppression de la PAJE

class paje(Variable):
    def function(self, simulation, period):
        '''
        Prestation d'accueil du jeune enfant
        '''
        period = period.this_month

        paje_base = simulation.calculate('paje_base', period)
        paje_naissance = simulation.calculate('paje_naissance', period)
        paje_clca = simulation.calculate('paje_clca', period)
        paje_clmg = simulation.calculate('paje_clmg', period)
        paje_colca = simulation.calculate('paje_colca', period)

        return period, (paje_base + (paje_naissance + paje_clca + paje_clmg + paje_colca) / 12) * 0


In [10]:
class ReformeRevenuDeBaseparCSG(Reform):
    name = u"Réforme Revenu de base par CSG"

    def apply(self):
        for variable in [aspa, rsa_socle, af, paje]:
            self.update_variable(variable)
            self.modify_legislation_json(modifier_function = modify_legislation_json)

# Tests

In [11]:
reform = ReformeRevenuDeBaseparCSG(tax_benefit_system)

In [12]:
parent1_base_ressources = 1000000 # base annuelle

In [13]:
scenario_ref_individu_seul = tax_benefit_system.new_scenario().init_single_entity(
    period = 2016,
    parent1 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent1_base_ressources,
        asi_aspa_base_ressources_individu = parent1_base_ressources / 12, # base ressources de l'aspa
        statut_marital = u'Célibataire',
        ),
    foyer_fiscal = dict(
        caseT = True,
        ),
    enfants = [
        dict(
            date_naissance = date(2010, 1, 1),
            ),
        dict(
            date_naissance = date(2014, 1, 1),
            ),
        dict(
            date_naissance = date(2015, 1, 1),
            ),
        ],
    )

In [14]:
simulation_ref_individu_seul = scenario_ref_individu_seul.new_simulation(debug = True)

In [15]:
#df_ref__aspa = simulation_ref_individu_seul.calculate('aspa')
#print round(df_ref__aspa,2), "par mois"

In [16]:
df_rsa = simulation_ref_individu_seul.calculate('rsa')
print round(df_rsa,2)

AssertionError: Requested period 2016 differs from 2016-01 returned by variable rsa

In [ ]:
df_af = simulation_ref_individu_seul.calculate('af')
print round(df_af,2), "par mois"

In [ ]:
df_paje = simulation_ref_individu_seul.calculate('paje')
print round(df_paje,2)

from openfisca_core import tools
print(tools.get_trace_tool_link(scenario_ref_individu_seul, ['paje'], api_url='http://api.openfisca.fr/', trace_tool_url='http://www.openfisca.fr/tools/trace'))

In [ ]:
scenario_rdb_individu_seul = reform.new_scenario().init_single_entity(
    period = "2016-08",
    parent1 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent1_base_ressources,
        asi_aspa_base_ressources_individu = parent1_base_ressources,
        statut_marital = u'Célibataire',
        ),
    foyer_fiscal = dict(
        caseT = True,
        ),
    enfants = [
        dict(
            date_naissance = date(2010, 1, 1),
            ),
        dict(
            date_naissance = date(2012, 1, 1),
            ),
        dict(
            date_naissance = date(2013, 1, 1),
            ),
        ],
    )

In [ ]:
simulation_rdb_individu_seul = scenario_rdb_individu_seul.new_simulation(debug = True)

Calculs de référence (revenu disponible, IR, RSA, CSG imposable/deductible, impot total) 

In [ ]:
df_rdb__aspa = simulation_rdb_individu_seul.calculate('aspa')
print df_rdb__aspa

In [ ]:
df_rsa = simulation_rdb_individu_seul.calculate('rsa')
print df_rsa

In [ ]:
df_af = simulation_rdb_individu_seul.calculate('af')
print df_af

In [ ]:
df_paje = simulation_rdb_individu_seul.calculate('paje')
print round(df_paje,2)

In [ ]:
df_aspa_ref = simulation_ref_individu_seul.calculate('aspa')
df_aah_ref = simulation_ref_individu_seul.calculate('aah')
df_caah_ref = simulation_ref_individu_seul.calculate('caah')
df_asi_ref = simulation_ref_individu_seul.calculate('asi')
df_rsa_ref = simulation_ref_individu_seul.calculate('rsa')
df_aefa_ref = simulation_ref_individu_seul.calculate('aefa')
df_api_ref = simulation_ref_individu_seul.calculate('api')
df_ass_ref = simulation_ref_individu_seul.calculate('ass')
df_psa_ref = simulation_ref_individu_seul.calculate('psa')
df_ppa_ref = simulation_ref_individu_seul.calculate('ppa')
df_mini_ref = simulation_ref_individu_seul.calculate('mini')

tot_mini_ref = df_aspa_ref + df_aah_ref + df_caah_ref + df_asi_ref + df_rsa_ref + df_aefa_ref + df_api_ref + df_ass_ref + df_ppa_ref
print "Montants annuels             mensuels"
print "aspa  ", df_aspa_ref
print "aah   ", df_aah_ref
print "caah  ", df_caah_ref
print "asi   ", df_asi_ref
print "rsa   ", df_rsa_ref, "   ", df_rsa_ref / 12
print "aefa  ", df_aefa_ref
print "api   ", df_api_ref
print "ass   ", df_ass_ref
print "psa   ", df_psa_ref
print "ppa   ", df_ppa_ref
print "Total ", tot_mini_ref[0]
print "Minima", df_mini_ref

if tot_mini_ref[0] == df_mini_ref[0]:
    print "   ok, égalité vérifiée  "
